In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [41]:

# Define data transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [42]:
# Define the data directory
data_dir = 'D:\IIT-D\Sem-4\JCD893-M.Tech Major Project Part-2\Anchor_Frame\Generated_Images\dataset_atleast_1_0_75'

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
print(class_names)

{'train': 11717, 'val': 738, 'test': 1441}
['attack', 'benign']


In [21]:
# Define the data directory
data_dir = 'D:\IIT-D\Sem-4\JCD893-M.Tech Major Project Part-2\Anchor_Frame\Generated_Images\dataset_50_0.75'

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
print(dataset_sizes)

class_names = image_datasets['test'].classes
print(class_names)

{'test': 6371}
['attack', 'benign']


In [43]:
# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    # Unfreeze the final classification layer
    if "fc" in name:  
        param.requires_grad = True
    else:
        param.requires_grad = False
# Modify the final layer to match the number of classes
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
# Use all parameters
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  


# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [44]:
# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")
torch.save(model.state_dict(), 'custom_cnn_model_atleast_1_0_75.pth')

Epoch 0/1
----------
train Loss: 0.3470 Acc: 0.8744
val Loss: 0.0781 Acc: 0.9743
Epoch 1/1
----------
train Loss: 0.3586 Acc: 0.8799
val Loss: 0.0420 Acc: 0.9905
Training complete!


In [45]:
# Load the trained model
model = models.resnet18(pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('custom_cnn_model_atleast_1_0_75.pth'))
model.eval()
model = model.to(device)

# Test the model on the test dataset
test_loader = dataloaders['test']
all_preds = []
all_labels = []

for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Calculate accuracy
test_accuracy = np.sum(all_preds == all_labels) / len(all_labels)
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.9771
